In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

out = widgets.Output()
out

Output()

# Implementation

In [21]:
cleaned_text_output_filename.value

'demo.csv'

In [3]:
tab = widgets.Tab()

dirty_file_upload = widgets.FileUpload(
    accept=".csv",
    multiple=False
)

cleaned_text_output_label = widgets.Label(
    "Output File Name:", layout=widgets.Layout(width='150px')
)

cleaned_text_output_filename = widgets.Text(
    value="cleaned.csv",
    layout=widgets.Layout(width='150px')
)
clean_button = widgets.Button(
    description="Normalize data",
    icon='play',
    tooltip='Normalize data'
)

clean_tab = widgets.VBox([
    widgets.HBox([dirty_file_upload, clean_button], layout=widgets.Layout(margin="10px")),
    widgets.HBox([cleaned_text_output_label, cleaned_text_output_filename])])

clean_file_upload = widgets.FileUpload(
    accept=".csv",
    multiple=False
)
start_button = widgets.Button(
    description='Generate docx',
    icon="play",
    tooltip='Generate docx',
)


In [4]:
def label_and_input(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.HBox([
        widgets.Label(label, layout=label_layout),
        widgets.Text(value=text_default_value)
    ])

In [5]:
initial_name_tags = widgets.VBox([
    label_and_input("First Name", text_default_value='First Name'),
    label_and_input("Middle Initials", text_default_value='Middle Initial(s)'),
    label_and_input("Last Name", text_default_value="Last Name")
])

In [6]:
initial_examples = widgets.VBox([
label_and_input("Initials for \"Xiang-Zhen\": ", text_default_value="X-Z"),
label_and_input("Initials for \"Jun Soo\": ", text_default_value="J-S"),
label_and_input("Initials for \"Baskin-Sommers\": ", text_default_value="B-S"),
label_and_input("Initials for \"van Rooij\": ", text_default_value="vR")
])

In [7]:
def label_and_vertical_input(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.VBox([
        widgets.Label(label, layout=label_layout),
        widgets.Text(value=text_default_value, input_layout=input_layout)
    ])

In [8]:
def label_and_textarea(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.VBox([
        widgets.Label(label, layout=label_layout),
        widgets.Textarea(value=text_default_value, input_layout=input_layout)
    ])

In [9]:
generate_column_names = widgets.VBox([
    label_and_input("Full Name: ", text_default_value="Compound Name + highest degree"),
    label_and_input("Role(s): ", text_default_value="Role(s)"),
    label_and_vertical_input("Affiliations (as a Python List):", 
                      text_default_value="['Affiliation 1 Department, Institution', 'Affiliation 2 Department, Institution','Affiliation 3 Department, Institution']",
                       input_layout=widgets.Layout(width='600px')),
])
                       

In [10]:
generate_contribution_priority_order = widgets.VBox([
    label_and_textarea("Roles priority: ", input_layout=widgets.Layout(height="200%"))
])

In [11]:
accordion = widgets.Accordion([initial_name_tags, initial_examples, generate_column_names, generate_contribution_priority_order])
accordion.set_title(0, "Column names for reading authors' name")
accordion.set_title(1, "Format of initials from giving examples")
accordion.set_title(2, "Column names for generating authorship documentation")
accordion.set_title(3, "Roles priority")
accordion.selected_index = None

In [12]:
def start_generation(b):
    
    import io
    from generate_docx import InitialsGenerator, DocGenerator
    
    mydict = clean_file_upload.value
    bytes_val = mydict[next(iter(mydict.keys()))]['content']

    f = io.BytesIO(bytes_val)
    df = pd.read_csv(f)
    
    initials_generator = InitialsGenerator()
    initials =  initials_generator.transform(df)
    
    doc_generator = DocGenerator()
    doc_generator.affiliation_tags = ['Affiliation 1 Department & Address']
    doc_generator.output_doc_filename = "test_demo.docx"
    doc_generator.role_tag = generate_column_names.children[1].children[0].value
    doc_generator.generate(df, initials['Initial'])

In [13]:
start_button.on_click(start_generation)

In [14]:
generate_configuration = widgets.Textarea(
    description="Configuration JSON"
)
generate_tab = widgets.VBox([
    widgets.HBox([clean_file_upload, start_button], layout=widgets.Layout(margin="10px")),
    accordion
])

In [15]:
tab.children = [clean_tab, generate_tab]
tab.set_title(0, "Data Cleanning")
tab.set_title(1, "Generate Docx")


In [16]:
title = widgets.HTML("<h1>Authorship Statement Auto Generator</h1>")

In [17]:
tab.layout=widgets.Layout(width="80%", height="450px", min_width="800px")

In [18]:
with out:
    display(
        widgets.VBox([
            title,
            tab
        ], layout=widgets.Layout(align_items="center"))
    )